CASE 1: NO EAVESDROPPER

In [1]:
pip install qiskit[visualization]

Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import *
import random

In [3]:
KEY_LENGTH = 500
random.seed(0)

In [5]:
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1) 
    alice_bits += str(randBit) 
    
print("The bits Alice is going to send are: " + alice_bits[:10] + "...")

The bits Alice is going to send are: 0110000101...


In [6]:

def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    bases_string = ""
    for i in range(num_of_bases):
        randBasis = random.randint(0, 1)
        if randBasis == 0:
            bases_string += "Z" 
        else:
            bases_string += "X"
            
    return bases_string

In [7]:
alice_bases = generate_random_bases(KEY_LENGTH) # Alice randomly chooses a basis for each bit.
    
print("The bases Alice is going to encode them in are: " + alice_bases[:10] + "...")

The bases Alice is going to encode them in are: XZZXZZZXXX...


In [8]:
def encode(bits, bases):
    """This function encodes each bit into the given basis."""
    
    encoded_qubits = []
    
    for bit, basis in zip(bits, bases): #The zip() function takes iterables, aggregates them in a tuple, and returns it.
        qc = QuantumCircuit(1, 1) # Create a quantum circuit for each qubit, 1 quantum bit & 1 classical bit.
        
        # Possible Cases
        if bit=="0" and basis == "Z":
            encoded_qubits.append(qc)
            
        elif bit=="1" and basis == "Z":
            qc.x(0) # Apply X Gate
            encoded_qubits.append(qc)

        elif bit=="0" and basis == "X":
            qc.h(0) # Apply H Gate
            encoded_qubits.append(qc)
            
        elif bit=="1" and basis == "X":
            qc.x(0) # Apply X Gate
            qc.h(0) # Apply H Gate
            encoded_qubits.append(qc)
    return (encoded_qubits)

In [10]:
encoded_qubits = encode(alice_bits, alice_bases)

Sending qubits

In [11]:
QUANTUM_CHANNEL = encoded_qubits

In [12]:
bob_bases = generate_random_bases(KEY_LENGTH)
print("The bases Bob is going to decode them in are: " + bob_bases[:10] + "...")

The bases Bob is going to decode them in are: ZZXZXXXXZZ...


In [13]:
def measure(qubits, bases):
        """This function measures each qubit in the corresponding basis chosen for it."""

        bits = "" # The results of measurements

        for qubit, basis in zip(qubits, bases):

            # Add measurement depending on basis
            if basis == "Z":
                qubit.measure(0, 0)
            elif basis == "X":
                qubit.h(0)
                qubit.measure(0, 0)

            # Execute on Simulator
            simulator = Aer.get_backend('qasm_simulator')
            result = execute(qubit, backend=simulator, shots=1).result()
            counts = result.get_counts()
            measured_bit = max(counts, key=counts.get) # Max doesn't matter for simulator since there is only one shot.

            bits += measured_bit
            
        return bits

In [14]:
qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
bob_bits = measure(qubits_received, bob_bases)

print("The first few bits Bob received are: " + bob_bits[:10] + "...")

The first few bits Bob received are: 1101100111...


Compare

In [15]:
CLASSICAL_CHANNEL = alice_bases

In [16]:
common_bases = [i for i in range(KEY_LENGTH) if CLASSICAL_CHANNEL[i]==bob_bases[i]]

print("The indices of the first 10 bases they share in common are: " + str(common_bases[:10]))

The indices of the first 10 bases they share in common are: [1, 7, 10, 11, 12, 14, 19, 23, 26, 27]


In [17]:
bob_bits = [bob_bits[index] for index in common_bases]

In [18]:
CLASSICAL_CHANNEL = common_bases

In [19]:
alice_bits = [alice_bits[index] for index in common_bases]

In [20]:
CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.

# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yes, Alice and Bob seem to have the same bits!")
else:
    print("At least one of the bits is different.")

Yes, Alice and Bob seem to have the same bits!
